## Problem 1: Probability Warmup (8 points)

### Description
Let’s say we have an exam question which consists of 20 yes/no questions. From past performance of similar students, a randomly chosen student will know the correct answer to \( N \sim \text{Binom}(20, 11/20) \) questions. Furthermore, we assume that the student will guess the answer with equal probability to each question they don’t know the answer to. Given \( N \), we define \( Z \sim \text{Binom}(20 - N, 1/2) \) as the number of correctly guessed answers. Define \( Y = N + Z \), i.e., \( Y \) represents the number of total correct answers.

We are interested in setting a deterministic threshold \( T \), where we pass a student at threshold \( T \) if \( Y \geq T \). Here \( T \in \{0, 1, 2, \dots, 20\} \).

#### Subproblems
1. (5 points) For each threshold \( T \), compute the probability that the student knows less than 10 correct answers given that the student passed, i.e., \( N < 10 \). Store the results in `problem11_probabilities`, a list of length 21.
2. (3 points) What is the smallest value of \( T \) such that if \( Y \geq T \), then we are 90% certain that \( N \geq 10 \)? Provide the answer in `problem12_T`.



In [9]:

from scipy.special import binom as binomial  # 导入二项式系数计算函数

# 定义学生知道某题正确答案的概率
p = 6 / 10  

# 定义学生知道 N 个正确答案的概率 P(N = k)
# 使用二项分布公式：P(N = k) = C(10, k) * p^k * (1-p)^(10-k)
p_N = lambda k: binomial(10, k) * ((1 - p) ** (10 - k)) * (p ** k)

# 定义分数线范围 T
T = list(range(11))  # T 的取值范围是 [0, 1, 2, ..., 10]

# 初始化列表用于存储概率
lp_passed = []  # 每个分数线 T 下通过考试的概率 P(Y ≥ T)
lp_passed_Nless = []  # 每个分数线 T 下，N < 5 且通过考试的概率 P(N < 5 ∩ Y ≥ T)

# 遍历所有分数线 T
for t in T:
    p_passed = 0  # 当前分数线下的通过概率 P(Y ≥ T)
    p_passed_Nless = 0  # 当前分数线下，N < 5 且通过的概率 P(N < 5 ∩ Y ≥ T)
    
    # 遍历所有满足 Y ≥ T 的总正确答案数 y
    for y in range(t, 11):  # y 从 t 到 10，表示通过分数线的所有情况
        # 遍历所有可能的 N 值
        for N in range(y + 1):  # N 的范围是 [0, y]
            # 计算学生知道 N 个答案并且总共答对 Y 个答案的概率
            # P(Y = y | N) = C(10-N, y-N) * (1/2)^(10-N)
            # 总概率 = P(N = N) * P(Y = y | N)
            p_N_correct = p_N(N) * (binomial(10 - N, (y - N)) * (1 / 2) ** (10 - N))
            
            # 如果 N < 5，累加 P(N < 5 ∩ Y ≥ T)
            if N < 5: 
                p_passed_Nless += p_N_correct
                
            # 累加通过考试的总概率 P(Y ≥ T)
            p_passed += p_N_correct
    
    # 将当前分数线下的概率存入列表
    lp_passed.append(p_passed)  # 存储 P(Y ≥ T)
    lp_passed_Nless.append(p_passed_Nless)  # 存储 P(N < 5 ∩ Y ≥ T)

# 计算条件概率 P(N < 5 | Y ≥ T) = P(N < 5 ∩ Y ≥ T) / P(Y ≥ T)
p_con = [i / j for i, j in zip(lp_passed_Nless, lp_passed)]
# 将结果转换为 Python 的 float 类型，并格式化小数点
p_con = [round(float(x), 4) for x in p_con]  # 转换为 float 并保留 6 位小数
# 打印结果
print("条件概率 P(N < 5 | Y ≥ T)：", p_con)



条件概率 P(N < 5 | Y ≥ T)： [0.1662, 0.1662, 0.1662, 0.1662, 0.1655, 0.1609, 0.1445, 0.1122, 0.0732, 0.0406, 0.0197]


In [10]:
# Part 2: Give an integer between 0 and 10 which is the answer to 2.
problem12_T = 8


---

### Problem 2: Random Variable Generation and Transformation

```markdown
## Problem 2: Random Variable Generation and Transformation (8 points)

### Description
The purpose of this problem is to show that you can implement your own sampler, built in the following steps:

#### Subproblems
1. (2 points) Implement a **Linear Congruential Generator (LCG)** that satisfies the Hull-Dobell theorem.
2. (2 points) Using the LCG, construct random numbers from the uniform \([0, 1]\) distribution.
3. (4 points) Using the uniform \([0, 1]\) random generator, generate samples from the probability density:
   \[
   p_0(x) = \frac{\pi}{2} |\sin(2\pi x)|, \quad x \in [0, 1].
   \]
   Use the Accept-Reject sampler with the uniform \([0, 1]\) distribution as the proposal distribution.



In [12]:
import math  # 导入数学库
from sympy import primefactors  # 用于分解素因子的函数（代码中未使用）

def problem4_LCG(size=None, seed=0):
    """
    实现线性同余生成器 (Linear Congruential Generator, LCG)。
    生成指定数量的伪随机数。
    
    参数：
    size : int，生成的伪随机数个数
    seed : int，随机数生成的起始种子值
    
    返回：
    out : list，生成的伪随机数列表
    """
    M = 2**31 - 1  # 模数，取值为大素数 2^31 - 1
    a = 48271      # 乘数，满足 Hull-Dobell 定理
    b = 0          # 增量，取值为 0
    
    numbers = []  # 用于存储生成的伪随机数
    current = seed  # 初始化当前值为种子值
    
    for _ in range(size):  # 迭代 size 次
        current = (a * current + b) % M  # 使用 LCG 公式生成下一个随机数
        numbers.append(current)  # 将生成的随机数添加到结果列表
    
    return numbers  # 返回生成的伪随机数序列


In [ ]:
def problem4_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    raw_numbers = generator(size, seed)
    
   
    uniform_numbers = [x / period for x in raw_numbers]
    
    return uniform_numbers


---

### Problem 3: Concentration of Measure

```markdown
## Problem 3: Concentration of Measure (8 points)

### Description
Concentration of measure refers to the phenomenon where the probability of large deviations decreases as the sample size increases. Below are two questions related to this concept.

#### Subproblems
1. (4 points) Which of the following exponentially concentrate, i.e., satisfy:
   \[
   P(Z - \mathbb{E}[Z] \geq \epsilon) \leq C_1 e^{-C_2 n \epsilon^2} \wedge C_3 e^{-C_4 n (\epsilon + 1)}.
   \]
   Options:
   - The empirical mean of i.i.d. sub-Gaussian random variables
   - The empirical mean of i.i.d. sub-Exponential random variables
   - The empirical mean of i.i.d. random variables with finite variance
   - The empirical variance of i.i.d. random variables with finite variance
   - The empirical variance of i.i.d. sub-Gaussian random variables
   - The empirical variance of i.i.d. sub-Exponential random variables
   - The empirical third moment of i.i.d. sub-Gaussian random variables
   - The empirical fourth moment of i.i.d. sub-Gaussian random variables
   - The empirical mean of i.i.d. deterministic random variables
   - The empirical tenth moment of i.i.d. Bernoulli random variables

2. (4 points) Which of the above options concentrate in the weaker sense:
   \[
   P(Z - \mathbb{E}[Z] \geq \epsilon) \leq \frac{C_1}{n \epsilon^2}.
   \]


